In [1]:
import pandas as pd 
import numpy as np 

In [9]:
p1=pd.read_excel('评级_1.xls',header=0)
p1=p1.iloc[1:,:]
p1=p1[['Scode','Anndats','Recomm_code','Star']]
p1

,Scode,Anndats,Recomm_code,Star
1,000001,2002-06-28,3,0
2,000001,2002-07-18,3,0
3,000001,2003-06-04,3,0
4,000001,2003-06-06,3,0
5,000001,2004-03-29,4,0
...,...,...,...,...
29996,000527,2011-03-21,1,1
29997,000527,2011-03-22,1,0/0
29998,000527,2011-04-13,1,0
29999,000527,2011-04-13,1,1/1


In [10]:
df=p1
for num in range(2,21):
    file_name='评级_'+str(num)+'.xls'
    data=pd.read_excel(file_name,header=0)
    data=data.iloc[1:,:]
    data=data[['Scode','Anndats','Recomm_code','Star']]
    df=pd.concat([df,data],axis=0)

In [12]:
df.shape

(593343, 4)

In [33]:
star=df.Star.str.rsplit(pat='/',expand=True)
star=pd.DataFrame(star)
star=star.fillna(0)
star


,0,1,2,3,4,5,6,7
1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
23339,0,0,0,0,0,0,0,0
23340,0,0,0,0,0,0,0,0
23341,0,0,0,0,0,0,0,0
23342,0,0,0,0,0,0,0,0


In [44]:
star=star.astype(int)
star=star.sum(axis=1)


In [46]:
df['Star']=star
df.head(20)

,Scode,Anndats,Recomm_code,Star
1,000001,2002-06-28,3,0
2,000001,2002-07-18,3,0
3,000001,2003-06-04,3,0
4,000001,2003-06-06,3,0
5,000001,2004-03-29,4,0
6,000001,2004-04-15,3,0
7,000001,2004-04-15,5,0
8,000001,2004-05-31,5,1
9,000001,2004-06-29,4,0
10,000001,2004-08-18,5,0


In [48]:
df.Recomm_code=df.Recomm_code.replace({'5':-1,'4':-2,'3':0,'2':1,'1':0})
df.head(20)

,Scode,Anndats,Recomm_code,Star
1,000001,2002-06-28,0.0,0
2,000001,2002-07-18,0.0,0
3,000001,2003-06-04,0.0,0
4,000001,2003-06-06,0.0,0
5,000001,2004-03-29,-2.0,0
6,000001,2004-04-15,0.0,0
7,000001,2004-04-15,-1.0,0
8,000001,2004-05-31,-1.0,1
9,000001,2004-06-29,-2.0,0
10,000001,2004-08-18,-1.0,0


In [49]:
df.to_csv('star.csv',index=False)

In [2]:
df_sum=pd.read_csv('df_sum.csv',index_col=False)
df_sum=df_sum.iloc[:,1:]
df_sum.permno=df_sum.permno.astype(object)
def shift_recom(df):
    df['recom_lag']=df.recom.shift(1)
    return df
df_sum=df_sum.groupby('permno',as_index=False).apply(shift_recom)
df_sum=df_sum.dropna()
df_sum.head()

,permno,ym,recom,recom_lag
1,1,Feb 2006,0,0.0
2,1,Mar 2006,5,0.0
3,1,Apr 2006,-1,5.0
4,1,May 2006,0,-1.0
5,1,Jun 2006,-1,0.0


In [3]:
def set_group(df):
    df=df.sort_values('recom_lag',ascending=False)
    return df.iloc[:20,:]
lg_1=df_sum.groupby('ym',as_index=False).apply(set_group).reset_index(drop=True)
lg_1['lag_time']=1
lg_1.to_csv('time_one.csv',index=False)

In [4]:
df_sum=pd.read_csv('df_sum.csv',index_col=False)
df_sum=df_sum.iloc[:,1:]
df_sum.permno=df_sum.permno.astype(object)
def roll_col(df):
    df['recom']=df.recom.rolling(2,min_periods=1).sum()
    return df
df_sum=df_sum.groupby('permno',as_index=False).apply(roll_col)
def shift_recom(df):
    df['recom_lag']=df.recom.shift(1)
    return df
df_sum=df_sum.groupby('permno',as_index=False).apply(shift_recom)
df_sum=df_sum.dropna()
def set_group(df):
    df=df.sort_values('recom_lag',ascending=False)
    return df.iloc[:20,:]
lg_2=df_sum.groupby('ym',as_index=False).apply(set_group).reset_index(drop=True)
lg_2['lag_time']=2
lg_2.to_csv('time_two.csv',index=False)

In [5]:
df_sum=pd.read_csv('df_sum.csv',index_col=False)
df_sum=df_sum.iloc[:,1:]
df_sum.permno=df_sum.permno.astype(object)
def roll_col(df):
    df['recom']=df.recom.rolling(3,min_periods=1).sum()
    return df
df_sum=df_sum.groupby('permno',as_index=False).apply(roll_col)
def shift_recom(df):
    df['recom_lag']=df.recom.shift(1)
    return df
df_sum=df_sum.groupby('permno',as_index=False).apply(shift_recom)
df_sum=df_sum.dropna()
def set_group(df):
    df=df.sort_values('recom_lag',ascending=False)
    return df.iloc[:20,:]
lg_3=df_sum.groupby('ym',as_index=False).apply(set_group).reset_index(drop=True)
lg_3['lag_time']=3
lg_3.to_csv('time_three.csv',index=False)

In [7]:
lg_3

,permno,ym,recom,recom_lag,lag_time
0,19,Apr 2006,7.0,20.0,3
1,510,Apr 2006,6.0,17.0,3
2,2,Apr 2006,2.0,15.0,3
3,516,Apr 2006,11.0,13.0,3
4,511,Apr 2006,14.0,13.0,3
...,...,...,...,...,...
3095,2308,Sep 2018,9.0,10.0,3
3096,2567,Sep 2018,9.0,10.0,3
3097,2444,Sep 2018,9.0,9.0,3
3098,2325,Sep 2018,10.0,9.0,3


array(['Apr 2006', 'Apr 2007', 'Apr 2008', 'Apr 2009', 'Apr 2010',
       'Apr 2011', 'Apr 2012', 'Apr 2013', 'Apr 2014', 'Apr 2015',
       'Apr 2016', 'Apr 2017', 'Apr 2018', 'Aug 2006', 'Aug 2007',
       'Aug 2008', 'Aug 2009', 'Aug 2010', 'Aug 2011', 'Aug 2012',
       'Aug 2013', 'Aug 2014', 'Aug 2015', 'Aug 2016', 'Aug 2017',
       'Aug 2018', 'Dec 2006', 'Dec 2007', 'Dec 2008', 'Dec 2009',
       'Dec 2010', 'Dec 2011', 'Dec 2012', 'Dec 2013', 'Dec 2014',
       'Dec 2015', 'Dec 2016', 'Dec 2017', 'Dec 2018', 'Feb 2006',
       'Feb 2007', 'Feb 2008', 'Feb 2009', 'Feb 2010', 'Feb 2011',
       'Feb 2012', 'Feb 2013', 'Feb 2014', 'Feb 2015', 'Feb 2016',
       'Feb 2017', 'Feb 2018', 'Jan 2007', 'Jan 2008', 'Jan 2009',
       'Jan 2010', 'Jan 2011', 'Jan 2012', 'Jan 2013', 'Jan 2014',
       'Jan 2015', 'Jan 2016', 'Jan 2017', 'Jan 2018', 'Jul 2006',
       'Jul 2007', 'Jul 2008', 'Jul 2009', 'Jul 2010', 'Jul 2011',
       'Jul 2012', 'Jul 2013', 'Jul 2014', 'Jul 2015', 'Jul 20